In [1]:
import network as net
from loader import load_train_data, load_test_data, export_data
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import time
from sklearn.model_selection import train_test_split, KFold

In [11]:
EPOCHS = 0
RS = None

train, valid = load_train_data('train.csv', valid_size=.2, random_state=RS)
# train = load_train_data('train.csv')

start = time.time()

test = load_test_data("test.csv")
netw = net.Network([784, 100, 200, 10], random_state=RS)

acc, cf = netw.SGD(train, .5, EPOCHS, 10, 3.2, return_results=True, test_data=valid,
                                    monitor_train_evaluate=False)
print("%s seconds" % (time.time()-start))
net.visualisation(acc, cf, EPOCHS)
# net.visualisation(acc_t, cf_t, EPOCHS)

# netw.SGD(train, 2.714, 30, 30)
# export_data("last_ver.csv", netw.predict(test))

NameError: name 'load_train_data' is not defined

In [3]:
fig, ax = plt.subplots()
ax.plot(range(0, EPOCHS), acc, color="red")
ax.plot(range(0, EPOCHS), acc_t, color="blue")
plt.show()

NameError: name 'acc_t' is not defined

In [3]:
np.mean(acc[10:])

0.9668035714285714

In [10]:
import pickle
import gzip
import os
from loader import load_train_data

data = os.path.join("D:\Documents_zelen\my-first-neurol-network", "data/mnist.pkl.gz")

with gzip.open(data, "rb") as f:
    training_data, validation_data, test_data = pickle.load(f, encoding="latin-1")



0.99609375


In [3]:
help(pickle.load)

Help on built-in function load in module _pickle:

load(file, *, fix_imports=True, encoding='ASCII', errors='strict', buffers=())
    Read and return an object from the pickle data stored in a file.
    
    This is equivalent to ``Unpickler(file).load()``, but may be more
    efficient.
    
    The protocol version of the pickle is detected automatically, so no
    protocol argument is needed.  Bytes past the pickled object's
    representation are ignored.
    
    The argument *file* must have two methods, a read() method that takes
    an integer argument, and a readline() method that requires no
    arguments.  Both methods should return bytes.  Thus *file* can be a
    binary file object opened for reading, an io.BytesIO object, or any
    other custom object that meets this interface.
    
    Optional keyword arguments are *fix_imports*, *encoding* and *errors*,
    which are used to control compatibility support for pickle stream
    generated by Python 2.  If *fix_imports* i